# lumped resonator + hamburgermon + hamburgermon + lumped resonator

In [7]:
import numpy as np
import numpy.linalg as alg

In [8]:
hbar = 1.054e-34
h = hbar*2*np.pi
qe = 1.609e-19
flux_quant = h/2/qe

# Unit conversions
MHz = 10.0**(-3)
GHz = 1.0
kHz = 10.0**(-6)
us = 10.0**3
ns = 1.0

In [9]:
# F
Crq1 = 13.8e-15 # res1 <-> pad11
Cr1  = 151.7e-15 # res1 <-> gnd
Cj1  = 20.3e-15 # pad11 <-> pad21
C11  = 64.8e-15 # pad11 <-> gnd
C21  = 65.6e-15 # pad21 <-> gnd

Crq2 = 9.40e-15 # res2 <-> pad12
Cr2  = 138.5e-15 # res2 <-> gnd
Cj2  = 22.7e-15 # pad12 <-> pad22
C12  = 41.8e-15 # pad12 <-> gnd
C22  = 41.1e-15 # pad22 <-> gnd

Cg = 9.25e-15 # pad21 <-> pad22

res1_freq = 6.7e9 # Hz
res2_freq = 7.7e9 # Hz

# H
Lr1 = 1/(2*np.pi * res1_freq)**2/Cr1
Lr2 = 1/(2*np.pi * res2_freq)**2/Cr2
# print(Lr * 1e9, 'nH')
Lj1 = 12e-9
Lj2 = 12e-9

c_matrix = np.array([
    [Crq1+Cr1,  -Crq1,          0,          0,          0,              0], 
    [-Crq1,     Crq1+Cj1+C11,   -Cj1,       0,          0,              0],
    [0,         -Cj1,           Cj1+Cg+C21, 0,          0,              -Cg],
    [0,         0,              0,          Crq2+Cr2,   -Crq2,          0],
    [0,         0,              0,          -Crq2,      Crq2+Cj2+C12,   -Cj2],
    [0,         0,              -Cg,          0,          -Cj2,         Cj2+Cg+C22],
    ])

inv_l_matrix = np.array([
    [1/Lr1, 0,      0,      0,      0,      0], 
    [0,     1/Lj1,  -1/Lj1, 0,      0,      0],
    [0,     -1/Lj1, 1/Lj1,  0,      0,      0],
    [0,     0,      0,      1/Lr2,  0,      0],
    [0,     0,      0,      0,      1/Lj2,  -1/Lj2],
    [0,     0,      0,      0,      -1/Lj2, 1/Lj2],
])

print('Check symmetric:', np.allclose(c_matrix, c_matrix.T), np.allclose(l_matrix, l_matrix.T))

Check symmetric: True True


### simultaneous diagonalization

In [10]:
inv_c_matrix = alg.inv(c_matrix)
evals_c, u1 = alg.eigh(inv_c_matrix)
# print(evals_c)
# print(u1)
u1 = np.array([u1[i]/np.sqrt(evals_c[i]) for i in range(len(evals_c))])
evals, evecs = alg.eigh(u1.transpose() @ inv_l_matrix @ u1)

ECs = 1e-9 * qe**2/2/h * evecs.transpose() @ inv_c_matrix @ evecs
EJs = 1e-9 * flux_quant**2/h/4/np.pi**2 * evecs.transpose() @ inv_l_matrix @ evecs
Zs = np.array([np.sqrt(8*ECs[i,i]/EJs[i,i]) for i in range(len(evals_c))])
gs = np.array([[4*ECs[i,j]/np.sqrt(Zs[i]*Zs[j]) for i in range(len(evals_c))] for j in range(len(evals_c))])

print('Modes (cols are evecs)\n', np.around(evecs, 3))
print('EC values (GHz)\n', ECs)
print('EJ values (GHz)\n', EJs)
print('impedances (unitless)\n', Zs)
print('couplings (GHz)\n', gs)

Modes (cols are evecs)
 [[-0.095  0.003 -0.005 -0.197  0.001 -0.976]
 [-0.008 -0.053  0.132  0.007 -0.99  -0.002]
 [ 0.085 -0.099 -0.203 -0.952 -0.03   0.185]
 [-0.204  0.125  0.936 -0.222  0.118  0.06 ]
 [ 0.938 -0.219  0.243  0.034  0.037 -0.1  ]
 [-0.248 -0.961  0.08   0.061  0.064  0.009]]
EC values (GHz)
 [[ 0.23703407 -0.07161194  0.04823364 -0.00402293  0.00758908 -0.01133378]
 [-0.07161194  0.34046161 -0.04732414  0.02726825 -0.00145545  0.00428078]
 [ 0.04823364 -0.04732414  0.15753967  0.00850969  0.00354482 -0.00844023]
 [-0.00402293  0.02726825  0.00850969  0.20721916  0.04634854 -0.01371953]
 [ 0.00758908 -0.00145545  0.00354482  0.04634854  0.21103916  0.00772458]
 [-0.01133378  0.00428078 -0.00844023 -0.01371953  0.00772458  0.12283632]]
EJ values (GHz)
 [[ 21.70142734  10.49403525  -7.79158507   1.5617997   -0.50453141
    1.89175879]
 [ 10.49403525   8.29235608   7.97594122  -1.14956601  -0.10280575
   -0.94023027]
 [ -7.79158507   7.97594122  47.87699445  -6.60067102 

In [11]:
# expected nodes from the circuit (indexed from 1)
RES1_NODE = 1
RES2_NODE = 4
Q1_NODES = [2, 3]
Q2_NODES = [5, 6]

res1_mode = np.argmax(np.abs(evecs[RES1_NODE-1,:])) # look for col with max in row (node) 1
print('res1 mode:', res1_mode)
assert np.abs(evecs[RES1_NODE-1,res1_mode]) > 0.90, f'res1 mode is bad ({np.abs(evecs[RES1_NODE-1,res1_mode])})'

res2_mode = np.argmax(np.abs(evecs[RES2_NODE-1,:])) # look for col with max in node 4
print('res2 mode:', res2_mode)
assert np.abs(evecs[RES2_NODE-1,res2_mode]) > 0.90, f'res2 mode is bad ({np.abs(evecs[RES2_NODE-1,res2_mode])})'

q1_mode = -1
q1_mode_node0 = np.argmax(np.abs(evecs[Q1_NODES[0]-1,:])) # look for col with max in node 2
q1_mode_node1 = np.argmax(np.abs(evecs[Q1_NODES[1]-1,:])) # look for col with max in node 3
# Get the differential mode
if np.sign(evecs[Q1_NODES[0]-1,q1_mode_node0]) == -(np.sign(evecs[Q1_NODES[1]-1,q1_mode_node0])):
    q1_mode = q1_mode_node0
elif np.sign(evecs[Q1_NODES[1]-1,q1_mode_node1]) == -(np.sign(evecs[Q1_NODES[0]-1,q1_mode_node1])):
    q1_mode = q1_mode_node1
else: assert False, 'could not find qubit 1 mode'
print('qubit 1 mode:', q1_mode)

q2_mode = -1
q2_mode_node0 = np.argmax(np.abs(evecs[Q2_NODES[0]-1,:])) # look for col with max in node 2
q2_mode_node1 = np.argmax(np.abs(evecs[Q2_NODES[1]-1,:])) # look for col with max in node 3
# Get the differential mode
if np.sign(evecs[Q2_NODES[0]-1,q2_mode_node0]) == -(np.sign(evecs[Q2_NODES[1]-1,q2_mode_node0])):
    q2_mode = q2_mode_node0
elif np.sign(evecs[Q2_NODES[1]-1,q2_mode_node1]) == -(np.sign(evecs[Q2_NODES[0]-1,q2_mode_node1])):
    q2_mode = q2_mode_node1
else: assert False, 'could not find qubit 1 mode'
print('qubit 2 mode:', q2_mode)

res1 mode: 5
res2 mode: 2
qubit 1 mode: 3
qubit 2 mode: 0


In [12]:
print('EC1:', ECs[q1_mode, q1_mode], 'EJ1:', EJs[q1_mode, q1_mode], 'wo:', np.sqrt(8*ECs[q1_mode,q1_mode]*EJs[q1_mode,q1_mode]))
print('EC2:', ECs[q2_mode, q2_mode], 'EJ2:', EJs[q2_mode, q2_mode], 'wo:', np.sqrt(8*ECs[q2_mode,q2_mode]*EJs[q2_mode,q2_mode]))
print('coupling q1 <-> res1 (MHz):', np.abs(gs[res1_mode, q1_mode])*1e3)
print('coupling q2 <-> res2 (MHz):', np.abs(gs[res2_mode, q2_mode])*1e3)
print('coupling q1 <-> q2 (MHz):', np.abs(gs[q1_mode, q2_mode])*1e3)

EC1: 0.20721916432350027 EJ1: 16.725094685063073 wo: 5.2655561103231525
EC2: 0.23703406737860752 EJ2: 21.701427335506146 wo: 6.41496848893668
coupling q1 <-> res1 (MHz): 250.49908943507754
coupling q2 <-> res2 (MHz): 880.9853286969854
coupling q1 <-> q2 (MHz): 52.74865730423706
